In [1579]:
import jupyter
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
from urllib.parse import urlparse
from sklearn.model_selection import train_test_split
from itertools import groupby
import random

log_file_path = 'C://Users//yashs//OneDrive//Desktop//ISRO Hackathon//topic05//topic05//apache_logs//ssl_access_log-20231107.log'

with open(log_file_path, 'r') as file:
    log_entries = file.readlines()

# parsed_logs = []
# for entry in log_entries:
#     parts = entry.split(' ')
#     log_dict = {
#         'ip_address': parts[0],
#         'timestamp': parts[3][1:] + ' ' + parts[4][:-1],
#         'request_line': ' '.join(parts[5:8]),
#         'status_code': parts[8],
#         'bytes_transferred': parts[9],
#         'referrer': parts[10],
#         'user_agent': ' '.join(parts[11:]),
#     }
#     parsed_logs.append(log_dict)
# 
# logs_df = pd.DataFrame(parsed_logs)

columns = ['ip_address', 'timestamp', 'method', 'request_line', 'http_version', 'status_code', 'bytes_transferred', 'referer', 'user_agent']
# columns = ['ip_address', 'timestamp', 'request_line', 'http_version', 'status_code', 'bytes_transferred', 'referer', 'user_agent']
log_list = []

for log_entry in log_entries:
    regex_pattern = r'(?P<ip_address>\d+\.\d+\.\d+\.\d+) - - \[(?P<timestamp>.*?)\] "(?P<method>\w+) (?P<request_line>.*?) (?P<http_version>HTTP/\d+\.\d+)" (?P<status_code>\d+) (?P<bytes_transferred>\d+) "(?P<referer>.*?)" "(?P<user_agent>.*?)"'
    # regex_pattern = r'(?P<ip_address>\d+\.\d+\.\d+\.\d+) - - \[(?P<timestamp>.*?)\] "(?P<method_request_line>\w+ \S+ .*?) (?P<http_version>HTTP/\d+\.\d+)" (?P<status_code>\d+) (?P<bytes_transferred>\d+) "(?P<referer>.*?)" "(?P<user_agent>.*?)"'
    match = re.match(regex_pattern, log_entry)
    if match:
        log_list.append(match.groupdict())

logs_df = pd.DataFrame(log_list, columns=columns)
logs_df['timestamp'] = pd.to_datetime(logs_df['timestamp'], format='%d/%b/%Y:%H:%M:%S %z')


# logs_df['parsed_url'] = pd.DataFrame([urlparse(logs_df['request_line'][x].split()[1]) for x in range(len(logs_df['request_line']))])['path']
# print(f"Path: {parsed_url.path}")
# print(f"Query: {parsed_url.query}")
logs_df['log_index'] = logs_df.index 
logs_df

,ip_address,timestamp,method,request_line,http_version,status_code,bytes_transferred,referer,user_agent,log_index
0,172.26.3.214,2023-11-06 00:00:02+05:30,GET,/scan_m/cas_userstats/getUserStats.php?sdate=2...,HTTP/1.1,200,93,-,python-requests/2.5.1 CPython/3.8.5 Linux/3.10...,0
1,172.26.3.200,2023-11-06 00:00:03+05:30,GET,/rbi,HTTP/1.1,301,248,-,Python-urllib/3.7,1
2,172.26.3.200,2023-11-06 00:00:03+05:30,GET,/rbi/,HTTP/1.1,200,9298,-,Python-urllib/3.7,2
3,172.26.3.200,2023-11-06 00:00:04+05:30,GET,/rbi/,HTTP/1.1,200,9298,-,Python-urllib/3.7,3
4,172.26.3.214,2023-11-06 00:01:02+05:30,GET,/scan_m/cas_userstats/getUserStats.php?sdate=2...,HTTP/1.1,200,93,-,python-requests/2.5.1 CPython/3.8.5 Linux/3.10...,4
...,...,...,...,...,...,...,...,...,...,...
38592,172.26.3.214,2023-11-06 23:55:02+05:30,GET,/scan_m/cas_userstats/getUserStats.php?sdate=2...,HTTP/1.1,200,98,-,python-requests/2.5.1 CPython/3.8.5 Linux/3.10...,38592
38593,172.26.3.214,2023-11-06 23:56:01+05:30,GET,/scan_m/cas_userstats/getUserStats.php?sdate=2...,HTTP/1.1,200,98,-,python-requests/2.5.1 CPython/3.8.5 Linux/3.10...,38593
38594,172.26.3.214,2023-11-06 23:57:01+05:30,GET,/scan_m/cas_userstats/getUserStats.php?sdate=2...,HTTP/1.1,200,98,-,python-requests/2.5.1 CPython/3.8.5 Linux/3.10...,38594
38595,172.26.3.214,2023-11-06 23:58:01+05:30,GET,/scan_m/cas_userstats/getUserStats.php?sdate=2...,HTTP/1.1,200,98,-,python-requests/2.5.1 CPython/3.8.5 Linux/3.10...,38595


In [1580]:
#using status code in each group , analysing request line with higfh perecent of unsuccessful in certain group  
status_df = logs_df.groupby(by = ['ip_address','status_code']).count()
# for idx, group in status_df:
#     group.grpoupby()
# status_df.count()
#analyse using user agents - python request 
status_df

timestamp  method  request_line  http_version  \
ip_address     status_code                                                  
172.17.53.23   200                 33      33            33            33   
172.26.3.200   200                696     696           696           696   
               301                 51      51            51            51   
               302                108     108           108           108   
               403                  1       1             1             1   
               404                  7       7             7             7   
172.26.3.214   200              12418   12418         12418         12418   
172.26.3.55    200                 64      64            64            64   
               301                  1       1             1             1   
               404                  3       3             3             3   
172.31.4.102   200                 15      15            15            15   
               404                  3       3             3             3   
172.31.4.127   200                234     234           234           234   
               404                  4       4             4             4   
172.31.4.134   200                616     616           616           616   
               301                  1       1             1             1   
172.31.4.188   200                 44      44            44            44   
               301                  1       1             1             1   
               302                  3       3             3             3   
172.31.4.190   200                 16      16            16            16   
               404                  2       2             2             2   
172.31.4.195   200              11976   11976         11976         11976   
               302                199     199           199           199   
               404                 28      28            28            28   
172.31.4.196   200                 50      50            50            50   
               302                142     142           142           142   
172.31.4.54    200                115     115           115           115   
               404                  2       2             2             2   
192.168.1.125  200                309     309           309           309   
               206                  1       1             1             1   
               301                 16      16            16            16   
               302                204     204           204           204   
               400                208     208           208           208   
               403                204     204           204           204   
               404               8552    8552          8552          8552   
               405                  2       2             2             2   
               417                  2       2             2             2   
               431                  2       2             2             2   
192.168.193.36 200               1893    1893          1893          1893   
               301                  1       1             1             1   
               302                 84      84            84            84   
               404                  1       1             1             1   
192.168.193.6  200                218     218           218           218   
               302                 67      67            67            67   

                            bytes_transferred  referer  user_agent  log_index  
ip_address     status_code                                                     
172.17.53.23   200                         33       33          33         33  
172.26.3.200   200                        696      696         696        696  
               301                         51       51          51         51  
               302                        108      108         108        108  
     

In [1581]:
# import argparse
# import json
# import pandas as pd
# import re
# # from typing import list
#  
# quiet=False
# 
# def print_message(message: str):
#     global quiet
#     if not quiet:
#         print(message)
# def get_matches(log_file:str, regex):
#     with open(log_file, 'r') as f:
#         line=f.readline()
#         while line:
#             line=line.strip()
#             matches=re.match(regex,line)
#             if not matches:
#                 print_message(f"Warning, unable to parse the log message : {line}")
#                 line=f.readline()
#                 continue
#             groups=matches.groups()
#             yield groups
#             line=f.readline()
# def parse_apache_logs(log_file: str)->list:
#     logs=[]
#     regex=re.compile(r'^(\d+\.\d+\.\d+\.\d+) - - \[(\d+/\w+/\d+:\d{2}:\d{2}:\d{2}\s\+?\d{4})\] "(\w+)?\s?[/]?([^\^]+)(HTTP[^"]+)"\s(\d+|-)\s(\d+|-)\s"([^\^]+|-)"\s"([^\^]+|-)"$')
#     #  - - \[(\d+/\w+/\d+):(\d{2}:\d{2}:\d{2})\s(\+?-?\d{4})\] "(\w+) /([^"]+)"\s(\d+)\s(\d+)\s"([^"]+)"\s"([^"]+)"$
#     for groups in get_matches(log_file,regex):
#         log_dict={'IP_address':groups[0],'Date':groups[1],'Timestamp':groups[2],'TimeOffset':groups[3],\
#                   'Command':groups[4],'Path_of_Requested_File':groups[5],'G6':groups[6], 'SRC':groups[7],\
#                     'Bytes_Received':groups[8],'Referral_URL':groups[9],'User_Agent':groups[10]}
#         # """,'Date':groups[1],'Timestamp':groups[2],'TimeOffset':groups[3],\
#         #           'Command':groups[4],'Path_of_Requested_File':groups[5],'SRC':groups[6],\
#         #           'Bytes_Received':groups[7],'Referral_URL':groups[8],'User_Agent':groups[9]}"""
#         logs.append(log_dict)
#     return logs
# a=parse_apache_logs('ssl_access_log-20231107')
# b=pd.DataFrame(a)
# b.to_csv('my_oldata.csv')

In [1582]:
#  ** Drop duplicates **
# #logs_df.drop_duplicates(keep=False)
# reduce_df = logs_df.drop_duplicates()
# print(logs_df.isnull().any(axis=0))
# reduce_df

In [1583]:
# referer_df = logs_df['referer'].drop_duplicates()
referer_df = logs_df['referer']
print(len(referer_df))
# referer_df.loc[referer_df == '-']
print(len(referer_df.loc[referer_df != '-']))
print(len(referer_df.loc[referer_df == '-']))
# referer_df

38597
16919
21678


In [1584]:
# status_list = []
# for i in logs_df['status_code']:
#     if (i not in status_list):
#         status_list.append(i)
# print(status_list)

## pd.to_datetime(logs_df['timestamp'], format="%d/%b/%Y:%H:%M:%S %z")
print(type(logs_df['timestamp'][1]))
print(logs_df['timestamp'])

# timestamp_list = []
# for timestamp in logs_df['timestamp'].tolist():
#     ## print(timestamp)
#     i=pd.to_datetime(str(timestamp),format= "%d/%b/%Y:%H:%M:%S %z")
#     timestamp_list.append(i)
#
# logs_df['timestamp'] = pd.DataFrame(timestamp_list)
##print(len(logs_df['timestamp']),len(pd.DataFrame(timestamp_list)))

# print(logs_df['timestamp'][1])
# print(type(logs_df['timestamp']))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
0       2023-11-06 00:00:02+05:30
1       2023-11-06 00:00:03+05:30
2       2023-11-06 00:00:03+05:30
3       2023-11-06 00:00:04+05:30
4       2023-11-06 00:01:02+05:30
                   ...           
38592   2023-11-06 23:55:02+05:30
38593   2023-11-06 23:56:01+05:30
38594   2023-11-06 23:57:01+05:30
38595   2023-11-06 23:58:01+05:30
38596   2023-11-06 23:59:02+05:30
Name: timestamp, Length: 38597, dtype: datetime64[ns, UTC+05:30]


In [1585]:
logs_df['timezone'] = logs_df['timestamp'].dt.tz
# print(logs_df['timezone'].drop_duplicates())
if len(logs_df['timezone'].drop_duplicates() == 1):
    logs_df['timestamp_localised'] = logs_df['timestamp'].dt.tz_localize(None)
# logs_df

In [1586]:
#session details - duration , frequecy, session start/end time
logs_df = logs_df.sort_values(by =['ip_address','timestamp_localised'])
logs_df

,ip_address,timestamp,method,request_line,http_version,status_code,bytes_transferred,referer,user_agent,log_index,timezone,timestamp_localised
23415,172.17.53.23,2023-11-06 12:33:41+05:30,GET,/scan_m/fog/,HTTP/2.0,200,8309,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,23415,UTC+05:30,2023-11-06 12:33:41
23416,172.17.53.23,2023-11-06 12:33:41+05:30,GET,/scan_m/fog/css/component.css,HTTP/2.0,200,1159,https://bhuvan-staging1.nrsc.gov.in/scan_m/fog/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,23416,UTC+05:30,2023-11-06 12:33:41
23417,172.17.53.23,2023-11-06 12:33:41+05:30,GET,/scan_m/fog/css/ol.css,HTTP/2.0,200,2313,https://bhuvan-staging1.nrsc.gov.in/scan_m/fog/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,23417,UTC+05:30,2023-11-06 12:33:41
23418,172.17.53.23,2023-11-06 12:33:41+05:30,GET,/scan_m/fog/theme/default/style2.css,HTTP/2.0,200,4345,https://bhuvan-staging1.nrsc.gov.in/scan_m/fog/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,23418,UTC+05:30,2023-11-06 12:33:41
23419,172.17.53.23,2023-11-06 12:33:41+05:30,GET,/scan_m/fog/theme/default/jquery-ui.min.css,HTTP/2.0,200,8343,https://bhuvan-staging1.nrsc.gov.in/scan_m/fog/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,23419,UTC+05:30,2023-11-06 12:33:41
...,...,...,...,...,...,...,...,...,...,...,...,...
26926,192.168.193.6,2023-11-06 14:26:51+05:30,POST,/wdc2.0/php/get_states.php,HTTP/2.0,200,355,https://bhuvan-staging1.nrsc.gov.in/wdc2.0/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,26926,UTC+05:30,2023-11-06 14:26:51
26927,192.168.193.6,2023-11-06 14:26:51+05:30,POST,/wdc2.0/php/get_heads.php,HTTP/2.0,200,329,https://bhuvan-staging1.nrsc.gov.in/wdc2.0/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,26927,UTC+05:30,2023-11-06 14:26:51
26928,192.168.193.6,2023-11-06 14:26:51+05:30,POST,/wdc2.0/php/get_states.php,HTTP/2.0,200,355,https://bhuvan-staging1.nrsc.gov.in/wdc2.0/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,26928,UTC+05:30,2023-11-06 14:26:51
26929,192.168.193.6,2023-11-06 14:26:51+05:30,POST,/wdc2.0/php/get_heads.php,HTTP/2.0,200,329,https://bhuvan-staging1.nrsc.gov.in/wdc2.0/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,26929,UTC+05:30,2023-11-06 14:26:51


In [1587]:
logs_df['sess_duration'] = logs_df.groupby('ip_address')['timestamp'].diff()
# logs_df 
timestamp_series = logs_df['timestamp'].sort_values(ascending=False)
# timestamp_series
logs_df['session'] = 1
print(type(logs_df['sess_duration'][1]),type(pd.to_datetime("00:30:00 +05:30",format= "%H:%M:%S %z")))
print(type(logs_df['sess_duration'][5]),type(timedelta(minutes=30)))
# print(type(np.datetime64('NaT')))
# print(type(np.NAN))
nat_type = type(logs_df['sess_duration'][1])
logs_df.loc[ (type(logs_df['sess_duration']) != type(logs_df['sess_duration'][1])) and
            (logs_df['sess_duration'] > timedelta(minutes=30)),
            'session'] = 0
# (logs_df['sess_duration'] != np.datetime64('NaT') )
# (logs_df['sess_duration'] > pd.to_datetime("00:30:00+05:30",format= "%H:%M:%S%z"))
# (logs_df['sess_duration'] > timedelta(minutes=30))

logs_df = logs_df.reset_index()

session_index = logs_df.columns.get_indexer_for(target=['session'])
#after these !=5 if only log_indices , same ip_address but log_index changes index doesnt change , the next log_index is of other ip_address
end_indices = logs_df.loc[logs_df['session'] == 0].index
print(end_indices)
print(logs_df.iloc[end_indices[0], session_index] )
for idx in end_indices:
    if ((logs_df.iloc[idx+1, session_index].item() != logs_df.iloc[idx, session_index].item() ) & 
            (logs_df.iloc[idx, session_index].item() == 0)):
        logs_df.iloc[idx+1, session_index] = 5 

end_log_indices = logs_df.loc[logs_df['session'] == 0]['log_index']
print(end_log_indices+1)
for idx in end_log_indices:
    # print(idx, logs_df.iloc[idx,:])
    # print(idx)
    if (logs_df.loc[logs_df['log_index'] == idx+1 ]['ip_address'].item() == 
            logs_df.loc[logs_df['log_index'] == idx ]['ip_address'].item() ):
        logs_df.loc[logs_df['log_index'] == idx+1 , 'session'] = 5 
    # if ():
    #     #can use at (rather than loc) - more fast 
print(logs_df.loc[logs_df['log_index'] == 22058 ]['ip_address'].item())
    
# print(logs_df.columns)
logs_df = logs_df[['log_index', 'ip_address', 'timestamp_localised', 'sess_duration', 'session', 'method', 'request_line','http_version', 'status_code', 'bytes_transferred', 'referer', 'user_agent', 'timezone','timestamp'
       ]]
logs_df
# logs_df.loc[logs_df['log_index'] == 22058 ]['ip_address']

<class 'pandas._libs.tslibs.nattype.NaTType'> <class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timedeltas.Timedelta'> <class 'datetime.timedelta'>
Index([  772,   848,   860, 13394, 13396, 13397, 13463, 13487, 13502, 13867,
       14320, 25871, 25872, 26465, 26509, 26779, 33637, 36541, 37850, 38310,
       38418],
      dtype='int64')
session    0
Name: 772, dtype: object
772      28133
848      34128
860      35348
13394    23357
13396    24802
13397    25694
13463    21773
13487    24847
13502    33109
13867    27558
14320    29738
25871    22058
25872    25461
26465    28081
26509    33117
26779    29892
33637    30172
36541    21931
37850    29144
38310    33039
38418    25855
Name: log_index, dtype: int64
172.26.3.214


,log_index,ip_address,timestamp_localised,sess_duration,session,method,request_line,http_version,status_code,bytes_transferred,referer,user_agent,timezone,timestamp
0,23415,172.17.53.23,2023-11-06 12:33:41,NaT,1,GET,/scan_m/fog/,HTTP/2.0,200,8309,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,UTC+05:30,2023-11-06 12:33:41+05:30
1,23416,172.17.53.23,2023-11-06 12:33:41,0 days 00:00:00,1,GET,/scan_m/fog/css/component.css,HTTP/2.0,200,1159,https://bhuvan-staging1.nrsc.gov.in/scan_m/fog/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,UTC+05:30,2023-11-06 12:33:41+05:30
2,23417,172.17.53.23,2023-11-06 12:33:41,0 days 00:00:00,1,GET,/scan_m/fog/css/ol.css,HTTP/2.0,200,2313,https://bhuvan-staging1.nrsc.gov.in/scan_m/fog/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,UTC+05:30,2023-11-06 12:33:41+05:30
3,23418,172.17.53.23,2023-11-06 12:33:41,0 days 00:00:00,1,GET,/scan_m/fog/theme/default/style2.css,HTTP/2.0,200,4345,https://bhuvan-staging1.nrsc.gov.in/scan_m/fog/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,UTC+05:30,2023-11-06 12:33:41+05:30
4,23419,172.17.53.23,2023-11-06 12:33:41,0 days 00:00:00,1,GET,/scan_m/fog/theme/default/jquery-ui.min.css,HTTP/2.0,200,8343,https://bhuvan-staging1.nrsc.gov.in/scan_m/fog/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,UTC+05:30,2023-11-06 12:33:41+05:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38592,26926,192.168.193.6,2023-11-06 14:26:51,0 days 00:03:50,1,POST,/wdc2.0/php/get_states.php,HTTP/2.0,200,355,https://bhuvan-staging1.nrsc.gov.in/wdc2.0/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,UTC+05:30,2023-11-06 14:26:51+05:30
38593,26927,192.168.193.6,2023-11-06 14:26:51,0 days 00:00:00,1,POST,/wdc2.0/php/get_heads.php,HTTP/2.0,200,329,https://bhuvan-staging1.nrsc.gov.in/wdc2.0/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,UTC+05:30,2023-11-06 14:26:51+05:30
38594,26928,192.168.193.6,2023-11-06 14:26:51,0 days 00:00:00,1,POST,/wdc2.0/php/get_states.php,HTTP/2.0,200,355,https://bhuvan-staging1.nrsc.gov.in/wdc2.0/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,UTC+05:30,2023-11-06 14:26:51+05:30
38595,26929,192.168.193.6,2023-11-06 14:26:51,0 days 00:00:00,1,POST,/wdc2.0/php/get_heads.php,HTTP/2.0,200,329,https://bhuvan-staging1.nrsc.gov.in/wdc2.0/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,UTC+05:30,2023-11-06 14:26:51+05:30


In [1588]:
#drop-off points
ip_end_df = logs_df.drop_duplicates(subset='ip_address',keep='last')
ip_start_df = logs_df.drop_duplicates(subset='ip_address',keep='first')
# ip_df = pd.concat([ip_end_df,ip_start_df],axis=0)
# ip_df.sort_values(by=['ip_address','timestamp_localised'])
# logs_df.loc[ip_start_df.iloc[:,0].index,['session']] = 'start'
# session_index = logs_df.columns.get_indexer_for(target=['session'])
# session_index = logs_df.columns.tolist().index('session')
# print(session_index,logs_df.columns)
ip_start_indices = ip_start_df['log_index']
for idx in ip_start_indices:
    logs_df.loc[logs_df['log_index'] == idx, 'session'] = 5
ip_end_indices = ip_end_df['log_index']
for idx in ip_end_indices:
    logs_df.loc[logs_df['log_index'] == idx, 'session'] = 0
# logs_df.loc[(logs_df['session'] == 0), ]
# ip_start_df
# logs_df['session']
# print(logs_df.iloc[idx+1,session_index])

In [1589]:
session_logs_df = logs_df.loc[(logs_df['session'] == 0) | (logs_df['session'] == 5) ]
session_start_df = logs_df.loc[(logs_df['session'] == 5) ]
session_end_df = logs_df.loc[(logs_df['session'] == 0) ]
print(len(session_end_df), len(session_start_df), len(session_logs_df))
session_logs_df
session_df = session_logs_df[['log_index','ip_address','timestamp_localised','request_line','session']]
print(session_df['timestamp_localised'].tolist()[1])
session_df = session_df.reset_index()
session_df['session_no.'] = session_df.index
# session_df['session_start'] = session_df['timestamp_localised']
# session_df['sesion_end'] = session_df['timestamp_localised'].shift(-1)
# Shifting within each group based on 'ip_address'
session_df['session_start'] = session_df.groupby('ip_address')['timestamp_localised'].transform(lambda x: x)
session_df['session_end'] = session_df.groupby('ip_address')['timestamp_localised'].transform(lambda x: x.shift(-1))
session_df['session_duration'] = session_df['session_end'] - session_df['session_start'] 
print(session_df['index'].tolist())
session_df

35 32 67
2023-11-06 12:33:42
[0, 32, 33, 772, 773, 848, 849, 860, 861, 895, 896, 13313, 13314, 13381, 13382, 13394, 13395, 13396, 13397, 13398, 13399, 13400, 13463, 13464, 13487, 13488, 13502, 13503, 13637, 13638, 13867, 13868, 14254, 14255, 14302, 14303, 14320, 14321, 25871, 25872, 25873, 26465, 26466, 26509, 26510, 26523, 26524, 26715, 26716, 26779, 26780, 26832, 26833, 33637, 33638, 36332, 36333, 36541, 36542, 37850, 37851, 38310, 38311, 38312, 38418, 38419, 38596]


,index,log_index,ip_address,timestamp_localised,request_line,session,session_no.,session_start,session_end,session_duration
0,0,23415,172.17.53.23,2023-11-06 12:33:41,/scan_m/fog/,5,0,2023-11-06 12:33:41,2023-11-06 12:33:42,0 days 00:00:01
1,32,23447,172.17.53.23,2023-11-06 12:33:42,/favicon.ico,0,1,2023-11-06 12:33:42,NaT,NaT
2,33,1,172.26.3.200,2023-11-06 00:00:03,/rbi,5,2,2023-11-06 00:00:03,2023-11-06 15:38:43,0 days 15:38:40
3,772,28132,172.26.3.200,2023-11-06 15:38:43,/rbi,0,3,2023-11-06 15:38:43,2023-11-06 15:38:43,0 days 00:00:00
4,773,28133,172.26.3.200,2023-11-06 15:38:43,/rbi/,5,4,2023-11-06 15:38:43,2023-11-06 18:01:12,0 days 02:22:29
...,...,...,...,...,...,...,...,...,...,...
62,38311,33039,192.168.193.36,2023-11-06 17:02:11,/wdc2.0/php/get_project_all.php,0,62,2023-11-06 17:02:11,NaT,NaT
63,38312,1073,192.168.193.6,2023-11-06 09:29:48,/wdc2.0/static/media/spritesheet.a4e0eb7ad904a...,5,63,2023-11-06 09:29:48,2023-11-06 14:10:35,0 days 04:40:47
64,38418,25854,192.168.193.6,2023-11-06 14:10:35,/wdc2.0/php/get_states.php,0,64,2023-11-06 14:10:35,2023-11-06 14:10:35,0 days 00:00:00
65,38419,25855,192.168.193.6,2023-11-06 14:10:35,/wdc2.0//php/get_userroles.php,5,65,2023-11-06 14:10:35,2023-11-06 14:26:51,0 days 00:16:16


In [1590]:
session_no = 0
logs_df['session_no.'] = 0
print(logs_df['session_no.'][2])
for i in range(len(logs_df)):
    logs_df['session_no.'][i] = session_no
    if (logs_df['session'][i].item() == 0):
        session_no += 1
total_sessions = (logs_df['session_no.'].describe()['max'])
avg_requests = int(len(logs_df)/total_sessions)
print(total_sessions, avg_requests)
session_stats = session_df.describe()
session_stats['session_duration']
logs_df

0


C:\Users\yashs\AppData\Local\Temp\ipykernel_36472\820173876.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  logs_df['session_no.'][i] = session_no


34.0 1135


,log_index,ip_address,timestamp_localised,sess_duration,session,method,request_line,http_version,status_code,bytes_transferred,referer,user_agent,timezone,timestamp,session_no.
0,23415,172.17.53.23,2023-11-06 12:33:41,NaT,5,GET,/scan_m/fog/,HTTP/2.0,200,8309,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,UTC+05:30,2023-11-06 12:33:41+05:30,0
1,23416,172.17.53.23,2023-11-06 12:33:41,0 days 00:00:00,1,GET,/scan_m/fog/css/component.css,HTTP/2.0,200,1159,https://bhuvan-staging1.nrsc.gov.in/scan_m/fog/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,UTC+05:30,2023-11-06 12:33:41+05:30,0
2,23417,172.17.53.23,2023-11-06 12:33:41,0 days 00:00:00,1,GET,/scan_m/fog/css/ol.css,HTTP/2.0,200,2313,https://bhuvan-staging1.nrsc.gov.in/scan_m/fog/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,UTC+05:30,2023-11-06 12:33:41+05:30,0
3,23418,172.17.53.23,2023-11-06 12:33:41,0 days 00:00:00,1,GET,/scan_m/fog/theme/default/style2.css,HTTP/2.0,200,4345,https://bhuvan-staging1.nrsc.gov.in/scan_m/fog/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,UTC+05:30,2023-11-06 12:33:41+05:30,0
4,23419,172.17.53.23,2023-11-06 12:33:41,0 days 00:00:00,1,GET,/scan_m/fog/theme/default/jquery-ui.min.css,HTTP/2.0,200,8343,https://bhuvan-staging1.nrsc.gov.in/scan_m/fog/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,UTC+05:30,2023-11-06 12:33:41+05:30,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38592,26926,192.168.193.6,2023-11-06 14:26:51,0 days 00:03:50,1,POST,/wdc2.0/php/get_states.php,HTTP/2.0,200,355,https://bhuvan-staging1.nrsc.gov.in/wdc2.0/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,UTC+05:30,2023-11-06 14:26:51+05:30,34
38593,26927,192.168.193.6,2023-11-06 14:26:51,0 days 00:00:00,1,POST,/wdc2.0/php/get_heads.php,HTTP/2.0,200,329,https://bhuvan-staging1.nrsc.gov.in/wdc2.0/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,UTC+05:30,2023-11-06 14:26:51+05:30,34
38594,26928,192.168.193.6,2023-11-06 14:26:51,0 days 00:00:00,1,POST,/wdc2.0/php/get_states.php,HTTP/2.0,200,355,https://bhuvan-staging1.nrsc.gov.in/wdc2.0/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,UTC+05:30,2023-11-06 14:26:51+05:30,34
38595,26929,192.168.193.6,2023-11-06 14:26:51,0 days 00:00:00,1,POST,/wdc2.0/php/get_heads.php,HTTP/2.0,200,329,https://bhuvan-staging1.nrsc.gov.in/wdc2.0/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,UTC+05:30,2023-11-06 14:26:51+05:30,34


In [1591]:
session_end_df = logs_df.loc[logs_df['session'] == 0].groupby(['session','request_line']).count()
print(len(session_end_df))
session_end_df

23


log_index  \
session request_line                                                    
0       /anganwadi/                                                 1   
        /bhutanforest/manifest.json                                 1   
        /bhuvan_new2d_support_apis/dop/getDop.php?y=22....          1   
        /bhuvan_new2d_support_apis/login/logout.php                 1   
        /datadownload/php/getLayerDataYears.php?theme=l...          1   
        /favicon.ico                                                1   
        /kyrdemo/index.html                                         2   
        /kyrdemo/manifest.webmanifest                               1   
        /lulc50k/static/media/spritesheet.a4e0eb7ad904a...          1   
        /nird_dashboard/dashboard_index.php                         5   
        /rbi                                                        3   
        /scan_alka/mhrd_ncert/sb/get/legend_hindi.php?q...          1   
        /scan_alka/mhrd_ncert/sb/sb-hi.php                          1   
        /scan_m/cas_userstats/getUserStats.php?sdate=20...          1   
        /scan_p/bhuvanNTL/geometryState.php?state=DELHI             1   
        /scan_p/bhuvanNTL/getStates.php                             2   
        /scan_p/bhuvanNTL/getStats.php?state=INDIA                  1   
        /twris/geoportal/css/images/ui-icons_ffffff_256...          1   
        /twris/geoportal/theme/default/images/ui-bg_gla...          1   
        /twris/geoportal/twris.php                                  3   
        /wdc2.0/dashboard/                                          1   
        /wdc2.0/php/get_project_all.php                             3   
        /wdc2.0/php/get_states.php                                  1   

                                                            ip_address  \
session request_line                                                     
0       /anganwadi/                                                  1   
        /bhutanforest/manifest.json                                  1   
        /bhuvan_new2d_support_apis/dop/getDop.php?y=22....           1   
        /bhuvan_new2d_support_apis/login/logout.php                  1   
        /datadownload/php/getLayerDataYears.php?theme=l...           1   
        /favicon.ico                                                 1   
        /kyrdemo/index.html                                          2   
        /kyrdemo/manifest.webmanifest                                1   
        /lulc50k/static/media/spritesheet.a4e0eb7ad904a...           1   
        /nird_dashboard/dashboard_index.php                          5   
        /rbi                                                         3   
        /scan_alka/mhrd_ncert/sb/get/legend_hindi.php?q...           1   
        /scan_alka/mhrd_ncert/sb/sb-hi.php                           1   
        /scan_m/cas_userstats/getUserStats.php?sdate=20...           1   
        /scan_p/bhuvanNTL/geometryState.php?state=DELHI              1   
        /scan_p/bhuvanNTL/getStates.php                              2   
        /scan_p/bhuvanNTL/getStats.php?state=INDIA                   1   
        /twris/geoportal/css/images/ui-icons_ffffff_256...           1   
        /twris/geoportal/theme/default/images/ui-bg_gla...           1   
        /twris/geoportal/twris.php                                   3   
        /wdc2.0/dashboard/                                           1   
        /wdc2.0/php/get_project_all.php                              3   
        /wdc2.0/php/get_states.php                                   1   

                                                            timestamp_localised  \
session request_line                                                              
0       /anganwadi/                                                           1   
        /bhutanforest/manifest.json                                           1   
        /bhuvan_new2d_support_apis/dop/getDop.php?y

In [1592]:
# logs_df.groupby(['ip_address','request_line'],axis=1).count()
# logs_df['count'] = 1
# count_df = logs_df.groupby(by =['request_line']).value_counts()

# #groupby count for ip_address - request_line
# count_df = logs_df.groupby(['ip_address','request_line']).count()
# count_df

In [1593]:
# #counts of unique request lines
# grouped = logs_df.groupby(['ip_address', 'request_line']).size().reset_index(name='count')
# grouped

In [1594]:
logs_df['request_line'] = logs_df['method'] + " " + logs_df['request_line']
logs_df.drop(labels = 'method', axis = 1,  inplace=True)
logs_df

,log_index,ip_address,timestamp_localised,sess_duration,session,request_line,http_version,status_code,bytes_transferred,referer,user_agent,timezone,timestamp,session_no.
0,23415,172.17.53.23,2023-11-06 12:33:41,NaT,5,GET /scan_m/fog/,HTTP/2.0,200,8309,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,UTC+05:30,2023-11-06 12:33:41+05:30,0
1,23416,172.17.53.23,2023-11-06 12:33:41,0 days 00:00:00,1,GET /scan_m/fog/css/component.css,HTTP/2.0,200,1159,https://bhuvan-staging1.nrsc.gov.in/scan_m/fog/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,UTC+05:30,2023-11-06 12:33:41+05:30,0
2,23417,172.17.53.23,2023-11-06 12:33:41,0 days 00:00:00,1,GET /scan_m/fog/css/ol.css,HTTP/2.0,200,2313,https://bhuvan-staging1.nrsc.gov.in/scan_m/fog/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,UTC+05:30,2023-11-06 12:33:41+05:30,0
3,23418,172.17.53.23,2023-11-06 12:33:41,0 days 00:00:00,1,GET /scan_m/fog/theme/default/style2.css,HTTP/2.0,200,4345,https://bhuvan-staging1.nrsc.gov.in/scan_m/fog/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,UTC+05:30,2023-11-06 12:33:41+05:30,0
4,23419,172.17.53.23,2023-11-06 12:33:41,0 days 00:00:00,1,GET /scan_m/fog/theme/default/jquery-ui.min.css,HTTP/2.0,200,8343,https://bhuvan-staging1.nrsc.gov.in/scan_m/fog/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,UTC+05:30,2023-11-06 12:33:41+05:30,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38592,26926,192.168.193.6,2023-11-06 14:26:51,0 days 00:03:50,1,POST /wdc2.0/php/get_states.php,HTTP/2.0,200,355,https://bhuvan-staging1.nrsc.gov.in/wdc2.0/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,UTC+05:30,2023-11-06 14:26:51+05:30,34
38593,26927,192.168.193.6,2023-11-06 14:26:51,0 days 00:00:00,1,POST /wdc2.0/php/get_heads.php,HTTP/2.0,200,329,https://bhuvan-staging1.nrsc.gov.in/wdc2.0/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,UTC+05:30,2023-11-06 14:26:51+05:30,34
38594,26928,192.168.193.6,2023-11-06 14:26:51,0 days 00:00:00,1,POST /wdc2.0/php/get_states.php,HTTP/2.0,200,355,https://bhuvan-staging1.nrsc.gov.in/wdc2.0/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,UTC+05:30,2023-11-06 14:26:51+05:30,34
38595,26929,192.168.193.6,2023-11-06 14:26:51,0 days 00:00:00,1,POST /wdc2.0/php/get_heads.php,HTTP/2.0,200,329,https://bhuvan-staging1.nrsc.gov.in/wdc2.0/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,UTC+05:30,2023-11-06 14:26:51+05:30,34


In [1722]:
#using markov chain models to predict next request line
from collections import defaultdict 

# Function to create Markov model for a session
def create_markov_model(session_logs):
    markov_model = defaultdict(dict)
    session_logs = session_logs.reset_index(drop=True)  # Resetting index for proper iteration
    for i in range(len(session_logs) - 1):
        current_request = session_logs['request_line'][i]
        next_request = session_logs['request_line'][i + 1]
        if next_request not in markov_model[current_request]:
            markov_model[current_request][next_request] = 1
        else:
            markov_model[current_request][next_request] += 1
    
    # Normalize transition probabilities
    for current_request, transitions in markov_model.items():
        total_transitions = sum(transitions.values())
        for next_request in transitions:
            markov_model[current_request][next_request] /= total_transitions
    
    return markov_model

def prediction_accuracy(session_logs, model):
    split_index = np.random.randint(len(session_logs)/2, len(session_logs))     #(inclusive, exclusive)
    
    session_accuracy=[]
    for i in range(split_index,len(session_logs)):
        first_part = session_logs['request_line'].tolist()[:i]  #(inclusive, exclusive)
        second_part = session_logs['request_line'].tolist()[i:] 
        # print(first_part)
        print(split_index,first_part)
        last_request = first_part[-1]
        # last_request = first_part[-1]['request_line'] if first_part else None
        next_request = model.get(last_request)
        # next_request = model.get(last_request) if last_request else None
    
        actual_next_request = second_part[0]
        # actual_next_request = second_part[0]['RequestLine'] if second_part else None
        accuracy = 1 if next_request == actual_next_request else 0 
        session_accuracy.append(accuracy)
    print(session_accuracy)
    return sum(session_accuracy)/len(session_accuracy)

# Construct Markov models for each session of each IP address
markov_models = defaultdict(dict)
#groupby for ip_address-session-requestline
grouped_sessions = logs_df.groupby(by=['ip_address','session_no.'])
unique_ips = logs_df['ip_address'].unique()  # Get unique IP addresses
session_logs_list = []
session_id_list = []
for session_id, session_logs in grouped_sessions:
    ip, session = session_id
    # print(f"IP: {ip}, Session: {session}")
    # session_id_list.append(np.array([ip,session]))
    session_id_list.append([ip,session])
    # print(session_logs)
    session_logs_list.append(session_logs)
# session_id_list = np.array(session_id_list)
# print(np.array(session_id_list).T[0])

# #again making group type
# gouped_sessions = pd.DataFrame({'ip_address' : [session[0] for session in session_id_list], 
#                                 'session_id' : [session[1] for session in session_id_list],
#                                 'session_data':session_logs_list})
# print(type(session_logs_list[0]))

# combined_data = list(zip(session_id_list, session_logs_list))

In [1723]:
indices = [i for i in range(len(session_id_list))]
random.shuffle(indices)
X_data = [session_id_list[i] for i in indices]
y_data = [session_logs_list[i] for i in indices]
X_train, X_test, y_train, y_test = [], [], [], []
print(int(0.1*len(session_id_list)))
for i in range(len(session_id_list)):
    if ((len(X_test) < int(0.2*len(session_id_list))) & (len(y_data[i]) < 0.3*avg_requests)): 
        # test_data.append(i)
        X_test.append(X_data[i])
        y_test.append(y_data[i])
    else:
        # train_data.append(i)
        X_train.append(X_data[i])
        y_train.append(y_data[i])

print(len(session_id_list),len(X_data),len(X_test),len(X_train))

3
35 35 7 28


In [1724]:
# X_train, X_test, y_train, y_test = train_test_split(session_id_list, session_logs_list
#                                                     , test_size= 0.1,random_state=42, shuffle=True)

In [1725]:
# # Sample list of tuples
# data = [(1, 'apple'), (1, 'banana'), (2, 'orange'), (2, 'grape'), (3, 'melon')]
# other_list = ['red', 'yellow', 'orange', 'purple', 'green']

# Sort the data based on the first indices of each tuple
sorted_X_train = sorted(X_train, key=lambda x: x[0])

# Group the sorted data by the first indices
grouped_X_train = {k: [item[1] for item in g] for k, g in groupby(sorted_X_train, key=lambda x: x[0])}

# Perform the same grouping for the other list using indices
grouped_y_train = {k: [y_train[idx] for idx in range(len(y_train)) if idx < len(v)] for k, v in grouped_X_train.items()}

print(grouped_X_train)
X_train = [key for key,value in grouped_X_train.items()]
print(X_train)
# type(grouped_y_train)
for i in grouped_y_train:
    pd.concat(grouped_y_train[i])
# print(grouped_y_train)

{'172.26.3.200': [2, 1, 3], '172.26.3.214': [5], '172.26.3.55': [6], '172.31.4.102': [9, 7], '172.31.4.127': [14, 12, 11], '172.31.4.134': [16, 15], '172.31.4.188': [17], '172.31.4.190': [18], '172.31.4.195': [21, 19, 20], '172.31.4.196': [24], '172.31.4.54': [25, 26], '192.168.1.125': [27, 28], '192.168.193.36': [31, 30, 29, 32], '192.168.193.6': [34, 33]}
['172.26.3.200', '172.26.3.214', '172.26.3.55', '172.31.4.102', '172.31.4.127', '172.31.4.134', '172.31.4.188', '172.31.4.190', '172.31.4.195', '172.31.4.196', '172.31.4.54', '192.168.1.125', '192.168.193.36', '192.168.193.6']


In [1726]:
for i in range(len(X_train)):
    ip = X_train[i]
    session_logs = y_train[i]
    # print(ip)
    markov_models[ip] = create_markov_model(session_logs)

In [1727]:
print(len(X_test))
X_test

7


[['172.31.4.127', 13],
 ['172.31.4.102', 10],
 ['172.17.53.23', 0],
 ['172.26.3.200', 4],
 ['172.31.4.102', 8],
 ['172.31.4.195', 23],
 ['172.31.4.195', 22]]

In [1728]:
# Measure accuracy for each session in validation data
total_sessions = 0
total_correct_predictions = 0

for i in range(len(X_test)):
    ip = X_test[i][0]
    session_id = X_test[i][1]
    session_logs = y_test[i]
    print(ip,session_id)
    if ip in markov_models:
        model = markov_models[ip]  # Get the trained Markov model for this session
        accuracy = prediction_accuracy(session_logs, model)
        print(accuracy)
        total_sessions += 1
        total_correct_predictions += accuracy

# Calculate overall accuracy
overall_accuracy = total_correct_predictions / total_sessions if total_sessions > 0 else 0

# for ip, sessions in val_data.items():
#     for session_id, session_logs in sessions.items():
#         if ip in markov_models and session_id in markov_models[ip]:
#             model = markov_models[ip][session_id]  # Get the trained Markov model for this session
#             accuracy = split_session_and_measure_accuracy(session_logs, model)
#             
#             total_sessions += 1
#             total_correct_predictions += accuracy

# # for using after enumerating for ip
# for ip in unique_ips:
#     ip_sessions = logs_df[logs_df['ip_address'] == ip]
#     for session_id, session_logs in ip_sessions.groupby('session_no.'):
#         print(session_logs)
#         markov_models[ip][session_id] = create_markov_model(session_logs)

# {'ip_address' : [session[0] for session in session_id_list], 
#                                 'session_id' : [session[1] for session in session_id_list],
#                                 'session_data':session_logs_list}
print('Overall accuracy is :',overall_accuracy)

172.31.4.127 13
9 ['GET /scan_p/bhuvanNTL/getStats.php?state=INDIA', 'GET /scan_p/bhuvanNTL/getFloodData.php', 'GET /scan_p/bhuvanNTL/getFloodData.php', 'GET /scan_p/bhuvanNTL/getStates.php', 'GET /scan_p/bhuvanNTL/getDistricts.php?statename=DELHI', 'GET /scan_p/bhuvanNTL/state_centroid.php?state=DELHI', 'GET /scan_p/bhuvanNTL/geometryState.php?state=DELHI', 'GET /scan_p/bhuvanNTL/indiageometry.php', 'GET /scan_p/bhuvanNTL/getStats.php?state=INDIA']
9 ['GET /scan_p/bhuvanNTL/getStats.php?state=INDIA', 'GET /scan_p/bhuvanNTL/getFloodData.php', 'GET /scan_p/bhuvanNTL/getFloodData.php', 'GET /scan_p/bhuvanNTL/getStates.php', 'GET /scan_p/bhuvanNTL/getDistricts.php?statename=DELHI', 'GET /scan_p/bhuvanNTL/state_centroid.php?state=DELHI', 'GET /scan_p/bhuvanNTL/geometryState.php?state=DELHI', 'GET /scan_p/bhuvanNTL/indiageometry.php', 'GET /scan_p/bhuvanNTL/getStats.php?state=INDIA', 'GET /scan_p/bhuvanNTL/geometryState.php']
9 ['GET /scan_p/bhuvanNTL/getStats.php?state=INDIA', 'GET /scan_p